## Using Keras Sequential API and Functional API

In [1]:
import os, random, numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

%matplotlib inline

C:\Anaconda3\envs\gpu\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Anaconda3\envs\gpu\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
C:\Anaconda3\envs\gpu\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# Load MNIST data

seed_value = 1234

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype("float32") / 255.0

# Sequential API

In [3]:
# Sequential API (Very convenient, not very flexible)

random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

model_1 = keras.Sequential(
    [
        keras.Input(shape=(x_train.shape[-1],)),
        layers.Dense(512, activation="relu"),
        layers.Dense(256, activation="relu"),
        layers.Dense(10),
    ]
)

model_1.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

model_1.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
print("\nloss = {:.4f}, accuracy = {:.4f}". format(*model_1.evaluate(x_test, y_test, batch_size=32, verbose=2)))

Epoch 1/5
1875/1875 - 5s - loss: 0.1857 - accuracy: 0.9429 - 5s/epoch - 3ms/step
Epoch 2/5
1875/1875 - 5s - loss: 0.0787 - accuracy: 0.9757 - 5s/epoch - 3ms/step
Epoch 3/5
1875/1875 - 5s - loss: 0.0546 - accuracy: 0.9827 - 5s/epoch - 3ms/step
Epoch 4/5
1875/1875 - 5s - loss: 0.0413 - accuracy: 0.9870 - 5s/epoch - 3ms/step
Epoch 5/5
1875/1875 - 5s - loss: 0.0338 - accuracy: 0.9889 - 5s/epoch - 3ms/step
313/313 - 0s - loss: 0.0699 - accuracy: 0.9803 - 389ms/epoch - 1ms/step

loss = 0.0699, accuracy = 0.9803


In [4]:
# Another way to use Sequential API

random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)

model_1b = keras.Sequential()
model_1b.add(keras.Input(shape=(x_train.shape[-1],)))
model_1b.add(layers.Dense(512, activation="relu", name="fc_layer1"))
model_1b.add(layers.Dense(256, activation="relu", name="fc_layer2"))
model_1b.add(layers.Dense(10, name="outputs"))

model_1b.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)

model_1b.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 fc_layer1 (Dense)           (None, 512)               401920    
                                                                 
 fc_layer2 (Dense)           (None, 256)               131328    
                                                                 
 outputs (Dense)             (None, 10)                2570      
                                                                 
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________


In [5]:
model_1b.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
print("\nloss = {:.4f}, accuracy = {:.4f}". format(*model_1b.evaluate(x_test, y_test, batch_size=32, verbose=2)))

Epoch 1/5
1875/1875 - 5s - loss: 0.1857 - accuracy: 0.9429 - 5s/epoch - 3ms/step
Epoch 2/5
1875/1875 - 5s - loss: 0.0787 - accuracy: 0.9757 - 5s/epoch - 3ms/step
Epoch 3/5
1875/1875 - 5s - loss: 0.0546 - accuracy: 0.9827 - 5s/epoch - 3ms/step
Epoch 4/5
1875/1875 - 5s - loss: 0.0413 - accuracy: 0.9870 - 5s/epoch - 3ms/step
Epoch 5/5
1875/1875 - 5s - loss: 0.0338 - accuracy: 0.9889 - 5s/epoch - 3ms/step
313/313 - 0s - loss: 0.0699 - accuracy: 0.9803 - 391ms/epoch - 1ms/step

loss = 0.0699, accuracy = 0.9803


# Functional API

In [6]:
# Functional API (A bit more flexible)

random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)

inputs = keras.Input(shape=(x_train.shape[-1],), name="inputs")
x = layers.Dense(512, activation="relu", name="fc_layer1")(inputs)
x = layers.Dense(256, activation="relu", name="fc_layer2")(x)
outputs = layers.Dense(10, activation="softmax", name="outputs")(x)

model_2 = keras.Model(inputs=inputs, outputs=outputs)

model_2.summary()

model_2.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

model_2.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
print("\nloss = {:.4f}, accuracy = {:.4f}". format(*model_2.evaluate(x_test, y_test, batch_size=32, verbose=2)))

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 784)]             0         
                                                                 
 fc_layer1 (Dense)           (None, 512)               401920    
                                                                 
 fc_layer2 (Dense)           (None, 256)               131328    
                                                                 
 outputs (Dense)             (None, 10)                2570      
                                                                 
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
1875/1875 - 5s - loss: 0.1857 - accuracy: 0.9429 - 5s/epoch - 3ms/step
Epoch 2/5
1875/1875 - 5s - loss: 0.0787 - accuracy: 0.9757 - 5s/epoch - 3ms/step
Epoch 3/5
1875/1875 - 5s 

# More on Functional API

In [7]:
# Functional API (A bit more flexible)

random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)

inputs = keras.Input(shape=(x_train.shape[-1],), name="inputs")
x1 = layers.Dense(512, activation="relu", name="fc_layer1")(inputs)
x2 = layers.Dense(256, activation="relu", name="fc_layer2")(x1)
x = layers.Concatenate(axis=1, name="concat_layer1")([x1, x2])
x = layers.Dense(64, activation="relu", name="fc_layer3")(x)
outputs = layers.Dense(10, activation="softmax", name="outputs")(x)

model_3 = keras.Model(inputs=inputs, outputs=outputs)

model_3.summary()

model_3.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

model_3.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
print("\nloss = {:.4f}, accuracy = {:.4f}". format(*model_3.evaluate(x_test, y_test, batch_size=32, verbose=2)))

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputs (InputLayer)            [(None, 784)]        0           []                               
                                                                                                  
 fc_layer1 (Dense)              (None, 512)          401920      ['inputs[0][0]']                 
                                                                                                  
 fc_layer2 (Dense)              (None, 256)          131328      ['fc_layer1[0][0]']              
                                                                                                  
 concat_layer1 (Concatenate)    (None, 768)          0           ['fc_layer1[0][0]',              
                                                                  'fc_layer2[0][0]']        